In [58]:
# Tablas para completar con PSD y Atterberg

# Project, Program y Location deben ser cargados casi completamente manualmente para PSD

# Location -> Podemos tener el TP-1 pero eso no es global y deberia ser usado
# en conjunto con programa y/o proyecto para ser utilizados como ID

# Sample -> Como ID podemos tener un UUID o utilizar el Sample ID que viene en el pdf
# Otra vez, el valor del PDF no el global por lo que deberiamos utilizar un ID compuesto
# Por otro lado el ensayo anterior utiliza UUIDs para eso, lo que me parece mejor
# From y To salen de los datos

# LabTest -> LabTestID (SRK-123-1234), el resto de los datos no estan en el informe


StatementMeta(, 73e84513-303a-44b1-8e97-abaecbeeff4d, 60, Finished, Available, Finished)

In [59]:
##
# Construye las tablas si es que no existen
##
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# PSD
psd_schema = StructType([
    StructField("LabTestID", StringType(), nullable=False),
    StructField("PSDType", StringType(), nullable=True),
    StructField("SieveSize_mm", StringType(), nullable=False),
    StructField("PercentPassing", StringType(), nullable=False),
])

# Atterberg
atterberg_schema = StructType([
    StructField("LabTestID", StringType(), nullable=False),
    StructField("LiquidLimit", StringType(), nullable=True),
    StructField("PlasticLimit", StringType(), nullable=True),
    StructField("PlasticiyIndex", StringType(), nullable=True),
    StructField("ContractionPerc", StringType(), nullable=True),
    StructField("Clasification", StringType(), nullable=True),
])

# GrainSize
grain_size_schema = StructType([
    StructField("LabTestID", StringType(), nullable=False),
    StructField("CobblePercent", FloatType(), nullable=True),
    StructField("GravelPercent", FloatType(), nullable=True),
    StructField("SandPercent", FloatType(), nullable=True),
    StructField("FinePercent", FloatType(), nullable=True),
    StructField("SiltPercent", FloatType(), nullable=True),
    StructField("ClayPercent", FloatType(), nullable=True),
])

# Crea las tablas si es que no existen ya
spark.createDataFrame([], psd_schema).write.format("delta").mode("ignore").saveAsTable("PSD")
spark.createDataFrame([], atterberg_schema).write.format("delta").mode("ignore").saveAsTable("Atterberg")
spark.createDataFrame([], grain_size_schema).write.format("delta").mode("ignore").saveAsTable("GrainSize")


StatementMeta(, 73e84513-303a-44b1-8e97-abaecbeeff4d, 61, Finished, Available, Finished)

In [60]:
##
# Leemos los datos que se van a procesar
##
import pandas as pd
from uuid import uuid4

# Se leen desde la tabla de bronce todos los datos que corresponden a esta fuente de datos
PSD_SOURCE_NAME = "SRK-169-Summary Sheet.pdf"

psd_df = spark.read.table("bronze_shortcut.data_PSD").where(f"Data_filename = '{PSD_SOURCE_NAME}'").toPandas()
atter_df = spark.read.table("bronze_shortcut.data_atterberg").where(f"Data_filename = '{PSD_SOURCE_NAME}'").toPandas()


def build_psd_records(full_psd, row_i):
    """Toma una de las filas de la tabla de bronce y la convierte 
    en un conjunto de valores de la tabla de plata"""
    # Solo queremos los valores
    sieve_sizes = list(set(full_psd.columns) - set(['Data_filename', 'Depth_m', 'GM', 'Lab_No', 'Sample']))
    df = full_psd.loc[:, sieve_sizes].iloc[row_i]

    # Transponemos y movemos el indice a una columna
    df = df.T.reset_index()

    # Renombramos las columnas para que matcheen con la tabla de PSD
    df = df.rename(columns={"index": "SieveSize_mm", row_i: "PercentPassing"})

    # Agregamos los valores que faltan
    df["LabTestID"] = full_psd.iloc[row_i]["Lab_No"]
    df["PSDType"] = ""

    return df


def build_PSD_table(df):
    """Convierte todos los datos de PSD en registros de la tabla de plata"""
    all_psd = [
        build_psd_records(df, i)
        for i in range(len(df))
    ]
    
    return pd.concat(all_psd, ignore_index=True)


def build_atterberg_table(df):
    """Convierte todos los datos de Atterberg en registros de la tabla de plata"""
    # Solo las columnas que nos interesan
    att = df.loc[:, ["Lab_No", "LL_%", "PI_%", "LS_%"]]

    # Son renombradas
    att = att.rename(columns={
        "Lab_No": "LabTestID", 
        "LL_%": "LiquidLimit", 
        "PI_%": "PlasticLimit", 
        "LS_%": "ContractionPerc"
    })

    # Agregamos las que faltan
    att["PlasticiyIndex"] = 0
    att["Clasification"] = ""
    
    return att


def _calc_depths(depths):
    """Calcula las profundidades "From", "Middle" y "To", las devuelve en tuplas"""

    # Si solo tengo una profundidad; son iguales
    if "-" not in depths:
        return depths.strip(), depths.strip(), depths.strip()
    
    # Tengo dos profundidades, calculo la media
    profundidades = [float(d.strip()) for d in depths.split("-")]
    prof_media = sum(profundidades)/2

    return str(profundidades[0]), str(prof_media), str(profundidades[1])


def build_tables(df):
    """Construye las tablas "LabTest" y "Sample" para la capa de plata"""
    # Solo nos insteresan ciertas columnas
    df = df.loc[:, ["Lab_No", "Sample", "Depth_m"]].rename(columns={"Lab_No": "LabTestID", "Sample": "Comment"})

    # Calculamos los valores que nos faltan
    df["SampleID"] = df.apply(lambda r: uuid4().hex, axis=1)
    df["DepthFrom_m"] = df.apply(lambda r: _calc_depths(r["Depth_m"])[0], axis=1)
    df["MiddleDepth_m"] = df.apply(lambda r: _calc_depths(r["Depth_m"])[1], axis=1)
    df["DepthTo_m"] = df.apply(lambda r: _calc_depths(r["Depth_m"])[2], axis=1)

    # Some empty but mandatory columns
    df["TestType"] = ""
    df["TestDate"] = None
    df["ReceivedDate"] = None
    df["LocationID"] = None
    df["SampleType"] = None
    df["MaterialType"] = None
    df["State"] = None
    df["Laboratory"] = None

    labtest = df.loc[:, [
        "LabTestID", "SampleID", "TestType", "Comment", 
        "TestDate", "ReceivedDate"
    ]]
    sample =  df.loc[:, [
        "SampleID", "DepthFrom_m", "MiddleDepth_m", "DepthTo_m", 
        "Comment", "LocationID", "SampleType", "MaterialType", 
        "State", "Laboratory"
    ]]

    return labtest, sample


PSD_table_data = build_PSD_table(psd_df)
Atterberg_table_data = build_atterberg_table(atter_df)

labtest_table, sample_table = build_tables(psd_df)


StatementMeta(, 73e84513-303a-44b1-8e97-abaecbeeff4d, 62, Finished, Available, Finished)

In [61]:
##
# Guardamos los datos en las tablas de plata
##

from delta.tables import DeltaTable


def guardar_datos(datos, nombre_tabla):
    """Guarga los datos en la tabla dada.
    Los datos duplicados no se guardan"""

    # Los datos son transformados a un Spark Dataframe
    nuevos_datos = spark.createDataFrame(datos)

    # Cuando los campos NO coinciden se insertan los datos
    DeltaTable.forName(spark, nombre_tabla).alias("old").merge(
        nuevos_datos.alias("new"),
        "old.LabTestID = new.LabTestID"
    ).whenNotMatchedInsertAll().execute()


def guardar_datos_sample(datos):
    """Guarga los datos en la tabla "Sample".
    Los datos duplicados no se guardan"""

    # Los datos son transformados a un Spark Dataframe
    nuevos_datos = spark.createDataFrame(datos)

    # Cuando los campos NO coinciden se insertan los datos
    DeltaTable.forName(spark, "Sample").alias("old").merge(
        nuevos_datos.alias("new"),
        "old.SampleID = new.SampleID"
    ).whenNotMatchedInsertAll().execute()


guardar_datos(PSD_table_data, "PSD")
guardar_datos(Atterberg_table_data, "Atterberg")
guardar_datos(labtest_table, "LabTest")


# La tabla de Sample tiene logica propia
guardar_datos_sample(sample_table)


StatementMeta(, 73e84513-303a-44b1-8e97-abaecbeeff4d, 63, Finished, Available, Finished)

In [62]:
%%sql
select * from sample limit 5

StatementMeta(, 73e84513-303a-44b1-8e97-abaecbeeff4d, 64, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 10 fields>